<a href="https://colab.research.google.com/github/nakamura196/tmp/blob/main/ndl_ocr_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NDLOCRチュートリアル2

Google Drive上のフォルダを指定して、当該フォルダに含まれる複数画像に対してOCR処理を実行し、指定したフォルダに認識結果を出力します。

参考：青池さんのノートブック

## 使用方法

- 「1.初期セットアップ」を実行してください。初回のみ実行が必要です。

- 「2. 設定」を変更してください。

- 「3. 実行」を行います。

## 1.初期セットアップ

（時間がかかります。初回のみ実行が必要です。）実行後は、「出力を消去」されることをお勧めします。

In [ ]:
# @title 初期セットアップ

%cd "/content/"

# 6. google driveのマウント
from google.colab import drive
drive.mount('/content/drive/')

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)
!git clone -q --recursive https://github.com/ndl-lab/ndlocr_cli

# 2. 必要なパッケージをインストールする
PROJECT_DIR="/content/ndlocr_cli"
!pip install -r {PROJECT_DIR}/requirements.txt
!pip install -q torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install -q mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
!pip install -q --upgrade numpy

%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
!pip install -q dist/*.whl
# %cd /content

# # 4. OCRに必要な学習済みモデルをダウンロードする

%cd {PROJECT_DIR}
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools

# 5. 環境変数を追加する
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

## 2. 設定

OCRの適用対象のファイルを設定します。

- input_dir: 画像が格納されたフォルダへのパス
- output_dir: 出力するフォルダへのパス
- extensions: 実行対象のファイル拡張子（,区切りで複数指定可）

`input_dir`と`output_dir`については、末尾に`/`の入力を忘れないようにしてください。

In [ ]:
#@title 設定
input_dir  = "/content/drive/MyDrive/ndl_ocr/input/" #@param {type:"string"}
output_dir  = "/content/drive/MyDrive/ndl_ocr/output/" #@param {type:"string"}
extensions = "jpeg,jpg" #@param {type:"raw"}

## 3. 実行

In [ ]:
#@title 実行

def createParentDir(path):
  basename = os.path.basename(path)
  parent_path = os.path.dirname(path)
  os.makedirs(parent_path, exist_ok=True)

import glob
def listFiles(input_dir, exts):
  paths = []
  ext_list = exts.split(",")
  for ext in ext_list:
    files = glob.glob("{}**/*.{}".format(input_dir, ext), recursive=True)
    paths.extend(files)

  return sorted(paths)

from tqdm import tqdm

files = listFiles(input_dir, extensions)

p = "2..3"

for file in tqdm(files):
  # 出力パスの作成
  output_path = "{}{}_p{}".format(output_dir, file.replace(input_dir, ""), p)

  if os.path.exists(output_path):
    !rm -rf $output_path

  createParentDir(output_path)
  !python main.py infer -s f $file $output_path -x -p $p

  basename_without_ext = os.path.splitext(os.path.basename(file))[0]

  target_dirs = ["pred_img", "txt", "xml"]

  for target_dir in target_dirs:
    input_tmp_path = "{}/{}/{}".format(output_path, basename_without_ext, target_dir)
    !mv $input_tmp_path/* $output_path

  !rm $output_path/opt.json
  !rm -rf $output_path/$basename_without_ext